In [ ]:
import pandas as pd
from sklearn.cross_validation import KFold
import numpy as np
import random

def error_rate(main_clf, pred_clf):
    calculate_pred = sum([1 for i in range(len(main_clf)) if main_clf[i] == pred_clf[i]])
    corr = calculate_pred / float(len(main_clf)) * 100.0
    clf_mis_error = 100 - corr

    return clf_mis_error

def c_error_rate(dataset, classifier, n_folds, learningRate, iteration):
    kf = KFold(len(dataset), n_folds=n_folds)
    main_error = []

    for train_index, test_index in kf:
        trainSet = dataset[train_index]
        testSet = dataset[test_index]
        predicted = classifier(trainSet, testSet, learningRate, iteration)
        actual = [dataRow[-1] for dataRow in testSet]
        errorRate = error_rate(actual, predicted)
        main_error.append(errorRate)

    return main_error

def clf_record(row, weights):
    output = weights[0]
    for i in range(len(row) - 1):
        output += weights[i + 1] * row[i]

    if output > 0.0:
        return 1
    else:
        return  - 1

def calculate_weight(trainData, learningRate, iteration):
    weights = [random.uniform(0,1) for i in range(len(trainData[0]))]
    for iter in range(iteration):
        for rowData in trainData:
            predictedClass = clf_record(rowData, weights)
            error = rowData[-1] - predictedClass
            weights[0] = weights[0] + learningRate * error
            for i in range(len(rowData) - 1):
                weights[i + 1] = weights[i + 1] + learningRate * error * rowData[i]
    return weights

def percep_clf(trainData, testData, learningRate, iteration):
    predictions = list()
    weights = calculate_weight(trainData, learningRate, iteration)
    for dataRow in testData:
        prediction = clf_record(dataRow, weights)
        predictions.append(prediction)
    return (predictions)

def data_set(fileName):
    dataset = pd.read_csv(fileName)
    if dataset['C'][0] == 'S' or dataset['C'][0] == 'NS':
        dataset['C'] = dataset['C'].map({'S': 1, 'NS': -1})
        
        
    dataset = dataset.as_matrix()
    return dataset

data_file = ['d-10.csv','d-100.csv']
learning_rate = np.random.randn((3,1))
iterations = [100,500]
n_folds = 4
main_error = []

for filename in data_file:
    print("Now showing results for file: "+ filename)
    dataset = data_set(filename)
    for learningRate in learning_rate:
        for iteration in iterations:
            errorRate = c_error_rate(dataset, percep_clf, n_folds, learningRate, iteration)
            print('Learning Rate = ', learningRate, 'Epoch = ', iteration)
            error_calculation = np.mean(errorRate)
            print('The error as the % of mis-classifications: ', error_calculation)
            main_error.append(error_calculation)
            
#references : internet 